In [24]:
%load_ext autoreload
%autoreload 2

from math import sqrt
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('/workspaces/pose'))
if module_path not in sys.path:
    sys.path.insert(0, '/workspaces/pose')

from pose import abelian_utils, fourier_utils3, hex_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
generators = np.array([[1, 0.5, 0], [0, sqrt(3)/2, 0], [0, 0, 1]])
n = 4
fourier_coef_shape = (n, n, n)
bf3 = abelian_utils.BumpFunction(np.array([0.0, 0.0, 0.0]), 0.03, generators, n)

In [3]:
coef3 = abelian_utils.unshuffle(bf3.func_dual.domain, bf3.values_dual, n, 1).reshape(fourier_coef_shape)
coef3_cropped = abelian_utils.crop_conjugate_symmetry(coef3, fourier_coef_shape)
coef3_flat = abelian_utils.flatten_coefs(coef3_cropped)
# coef3_final = np.delete(coef3_flat, 1, 0)

In [4]:
dim = 2 * (fourier_coef_shape[0] * fourier_coef_shape[1] * (fourier_coef_shape[2]//2+1) - 1)  # constant and purely real first coefficient (imaginary part of first coefficient is always 0)

res_u = 8
res_v = 8
res_w = 8
neuron_shape = (res_u, res_v, res_w)

gauss0_f_cropped_flat = coef3_flat

encoders = np.zeros((res_u*res_v*res_w, dim))
for mu in range(res_u):
    for mv in range(res_v):
        for mw in range(res_w):
            # assign encoders from (0,0,0) to (1,1,1) in row-major order (C-style)
            # center = np.array([u[mu, mv, mw], v[mu, mv, mw], w[mu, mv, mw]]) - np.array([0.5, np.sqrt(3)/4, 0.5])
            center = np.array([mu, mv, mw]) / np.array(neuron_shape)
            rot_mat = fourier_utils3._rot_mat_complete(fourier_coef_shape, center)
            coefs_cropped_flat = rot_mat.dot(gauss0_f_cropped_flat)
            encoders[mu*res_v*res_w+mv*res_w+mw, :] = coefs_cropped_flat[2:]

In [5]:
encoders.shape

(512, 94)

In [90]:
res3 = abelian_utils.unshuffle(bf3.func_ps.domain, bf3.coords_primal, n, 3)*n

(64, 3)

In [74]:
def get_3d_coordinates_unwrapped(neuron_shape, generators):
    res = np.zeros((*neuron_shape, 3))

    for u in range(neuron_shape[0]):
        for v in range(neuron_shape[1]):
            for w in range(neuron_shape[2]):
                res[u,v,w] = generators.dot(np.array([u, v, w]))

    return res

In [75]:
res4 = get_3d_coordinates_unwrapped(fourier_coef_shape, generators).reshape((-1,3))

In [89]:
np.array_equal(res3, res4)

True

In [80]:
u,v,w = hex_utils.get_3d_coordinates(n,n,n)
positions = np.vstack([u.ravel(), v.ravel(), w.ravel()]).transpose().reshape(n,n,n,3)
positions*n

array([[[[0.        , 0.        , 0.        ],
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 2.        ],
         [0.        , 0.        , 3.        ]],

        [[0.5       , 0.8660254 , 0.        ],
         [0.5       , 0.8660254 , 1.        ],
         [0.5       , 0.8660254 , 2.        ],
         [0.5       , 0.8660254 , 3.        ]],

        [[0.        , 1.73205081, 0.        ],
         [0.        , 1.73205081, 1.        ],
         [0.        , 1.73205081, 2.        ],
         [0.        , 1.73205081, 3.        ]],

        [[0.5       , 2.59807621, 0.        ],
         [0.5       , 2.59807621, 1.        ],
         [0.5       , 2.59807621, 2.        ],
         [0.5       , 2.59807621, 3.        ]]],


       [[[1.        , 0.        , 0.        ],
         [1.        , 0.        , 1.        ],
         [1.        , 0.        , 2.        ],
         [1.        , 0.        , 3.        ]],

        [[1.5       , 0.8660254 , 0.        ],
 

In [101]:
def unwrap_3d_coordinates(wrapped_coords, neuron_shape):
    res = np.zeros((*neuron_shape, 3))

    for u in range(neuron_shape[0]):
        for v in range(neuron_shape[1]):
            for w in range(neuron_shape[2]):
                u_corrected = u
                if v > 1:
                    u_corrected = (u+(v//2)) % neuron_shape[0]  # // is crucial
                res[u,v,w] = wrapped_coords[u_corrected, v, w]
                if v > 1 and u_corrected < (u+(v//2)):
                    res[u,v,w] += np.array([1.0, 0, 0])

    return res

In [144]:
def unwrap_single(wrapped, neuron_shape, d):
    if d == 1:
        res = np.zeros(neuron_shape)
    else:
        res = np.zeros((*neuron_shape, d))

    for u in range(neuron_shape[0]):
        for v in range(neuron_shape[1]):
            for w in range(neuron_shape[2]):
                u_corrected = u
                if v > 1:
                    u_corrected = (u+(v//2)) % neuron_shape[0]
                res[u,v,w] = wrapped[u_corrected, v, w]

    return res

In [145]:
def unwrap_multi(wrapped, neuron_shape, d):
    if d == 1:
        res = np.zeros((*neuron_shape, *neuron_shape))
    else:
        res = np.zeros((*neuron_shape, *neuron_shape, d))

    for u in range(neuron_shape[0]):
        for v in range(neuron_shape[1]):
            for w in range(neuron_shape[2]):
                u_corrected = u
                if v > 1:
                    u_corrected = (u+(v//2)) % neuron_shape[0]
                res[u,v,w] = unwrap_single(wrapped[u_corrected, v, w], neuron_shape, d)

    return res

In [102]:
res5 = (unwrap_3d_coordinates(positions, fourier_coef_shape)*n).reshape((-1,3))

In [109]:
np.allclose(res3, res5, rtol=1e-16, atol=1e-16)

True

In [119]:
u1, v1, w1 = hex_utils.get_3d_distances(*fourier_coef_shape)
res6 = np.vstack([u1.ravel(), v1.ravel(), w1.ravel()]).transpose().reshape(n*n,n*n,n*n,3)

In [120]:
res6.shape

(16, 16, 16, 3)

In [128]:
res7 = np.stack([u1, v1, w1], axis=-1).reshape(n,n,n,n,n,n,3)

In [129]:
res8 = unwrap_multi(res7, fourier_coef_shape)

In [133]:
res8[0,0,0,:,:,0]

array([[[ 0.        ,  0.        ,  0.        ],
        [ 0.125     ,  0.21650635,  0.        ],
        [-0.25      , -0.4330127 ,  0.        ],
        [-0.125     , -0.21650635,  0.        ]],

       [[ 0.25      ,  0.        ,  0.        ],
        [ 0.375     ,  0.21650635,  0.        ],
        [ 0.        , -0.4330127 ,  0.        ],
        [ 0.125     , -0.21650635,  0.        ]],

       [[ 0.5       ,  0.        ,  0.        ],
        [-0.375     ,  0.21650635,  0.        ],
        [ 0.25      , -0.4330127 ,  0.        ],
        [ 0.375     , -0.21650635,  0.        ]],

       [[-0.25      ,  0.        ,  0.        ],
        [-0.125     ,  0.21650635,  0.        ],
        [ 0.        ,  0.4330127 ,  0.        ],
        [-0.375     , -0.21650635,  0.        ]]])

In [146]:
mat_weights = hex_utils.create_rec_con_weights_hex(*fourier_coef_shape)
res9 = mat_weights.reshape(n,n,n,n,n,n)

In [147]:
res10 = unwrap_multi(res9, fourier_coef_shape, 1)

In [151]:
res10[0,0,0,:,:,0]

array([[1.32790009, 0.75093614, 0.1073035 , 0.75093614],
       [0.75093614, 0.22061896, 0.22061896, 0.75093614],
       [0.1073035 , 0.22061896, 0.1073035 , 0.22061896],
       [0.75093614, 0.75093614, 0.22061896, 0.22061896]])